# Extracting Metadata with Llamaindex

---
CONCLUSIONS:
* Overall, the extraction process was effective with summaries and keywords generated correctly, with 10 keywords as anticipated, but there were a few issues:
*	the abstract was included with the document title
*	some nodes produced more than 2 expected questions without answers instead of the expected 2 questions with answers
* Runtime extracting from 7 nodes (2 pages) was ~3 min.

* Custom Metadata Extractor modules can be crated. This has not been investigated in current notebook.
* Pydantic Extractor seems to work well with OpenAIPydanticProgram, however swiching to the local GuidancePydanticProgram breaks the pipline (https://github.com/run-llama/llama_index/issues/9914)
* GuidancePydanticProgram without the pipeline dones not output structured enough data:  OutputParserException: Failed to parse pydantic object from guidance program. Probably the LLM failed to produce data with right json schema. Most probably caused by truncation of output. max_len och max_tokens should be set higher, this was however not investigated in the current notebook.

* NOTE: version for llama-cpp-python was upgraded to latest when installing llama-index-llms-llama-cpp. Latest version might give unstable output for {Guidance}. Stable output with llama-cpp-python==0.2.26
* llama-cpp-python with GPU:\
CUDACXX=/usr/local/cuda-12/bin/nvcc CMAKE_ARGS="-DLLAMA_CUBLAS=on -DCMAKE_CUDA_ARCHITECTURES=all-major" FORCE_CMAKE=1 pip install llama-cpp-python==0.2.26 --no-cache-dir --force-reinstall --upgrade
---
---

In [ ]:
# !wget -O "data/10k-132.pdf" "https://www.dropbox.com/scl/fi/6dlqdk6e2k1mjhi8dee5j/uber.pdf?rlkey=2jyoe49bg2vwdlz30l76czq6g&dl=1"
# !wget -O "data/10k-vFinal.pdf" "https://www.dropbox.com/scl/fi/qn7g3vrk5mqb18ko4e5in/lyft.pdf?rlkey=j6jxtjwo8zbstdo4wz3ns8zoj&dl=1"

## Metadata Extractors modules
https://docs.llamaindex.ai/en/stable/examples/metadata_extraction/MetadataExtractionSEC/ \
https://docs.llamaindex.ai/en/stable/module_guides/indexing/metadata_extraction/ \
https://docs.llamaindex.ai/en/stable/examples/metadata_extraction/EntityExtractionClimate/ - Entitiy extractor

In [ ]:
import nest_asyncio # required for ingesion pipline
nest_asyncio.apply()

from llama_index.llms.llama_cpp import LlamaCPP

from llama_index.core.schema import MetadataMode
from llama_index.core.extractors import (
    SummaryExtractor,
    QuestionsAnsweredExtractor,
    TitleExtractor,
    KeywordExtractor,
    #BaseExtractor,
)
from llama_index.extractors.entity import EntityExtractor

from llama_index.core.node_parser import TokenTextSplitter
from llama_index.core import SimpleDirectoryReader
from llama_index.core.ingestion import IngestionPipeline


In [ ]:
llm = LlamaCPP(
    model_path="/home/dorota/models/mistral-7b-instruct-v0.2.Q6_K.gguf",
    context_window=16384, # n_ctx=0
    max_new_tokens=1024,
    model_kwargs={"n_gpu_layers": 33},
    verbose=False
) 

In [ ]:
text_splitter = TokenTextSplitter(
    separator=" ", chunk_size=512, chunk_overlap=128
)

In [ ]:
extractors = [
    TitleExtractor(nodes=5, llm=llm),
    QuestionsAnsweredExtractor(questions=2, llm=llm),
    EntityExtractor(prediction_threshold=0.5), # default model is tomaarsen/span-marker-mbert-base-multinerd
    SummaryExtractor(summaries=["prev", "self"], llm=llm),
    KeywordExtractor(keywords=10, llm=llm),
    # CustomExtractor()
]

In [ ]:
transformations = [text_splitter] + extractors

In [ ]:
ARTICLE = SimpleDirectoryReader(input_files=["/home/dorota/LLM-diploma-project/concept_tests/articles/40001_2023_Article_1364.pdf"]).load_data()
ARTICLE = ARTICLE[0:2]

In [ ]:
pipeline = IngestionPipeline(transformations=transformations)
nodes = pipeline.run(documents=ARTICLE)

In [ ]:
nodes[3].metadata

# extract from node 2 originating from  page 1
# {'page_label': '1',
#  'file_name': '40001_2023_Article_1364.pdf',
#  'file_path': '/home/dorota/LLM-diploma-project/concept_tests/articles/40001_2023_Article_1364.pdf',
#  'file_type': 'application/pdf',
#  'file_size': 4119241,
#  'creation_date': '2024-05-16',s
#  'last_modified_date': '2024-04-02',
#  'document_title': '\nBibliometric Analysis of Breast Cancer and Protein Synthesis Research: Current Trends and Future Directions\n
#                         Abstract:\nThis study aimed to provide an overview of the current state of research on breast cancer and protein synthesis using bibliometric analysis. We conducted a comprehensive search in Scopus database for articles published between 2015 and 2022 with the keywords “breast cancer,” “protein synthesis,” and related terms, including “translation,” “expression,” “cancer proteins,” and “proteomics.” The results were analyzed based on various metrics, including publication trends, journal impact factors, most productive countries, authors, and keywords. Our analysis revealed that there has been a significant increase in research publications on breast cancer and protein synthesis over the past few years. The most productive journals in this field were Journal of Biological Chemistry, Cancer Research, Proceedings of the National Academy of Sciences of the United States of America, and Oncogene. Keyword analysis showed that “breast cancer,” “expression,” “cancer,” “protein,” and “translation” were the most commonly researched topics. Our bibliometric analysis underscores the importance of continued research in this area to gain a better understanding of the biology of breast cancer and develop new therapies.\nIntroduction:\nBreast cancer is one of the leading causes of death among women worldwide, with an estimated 2.3 million new cases diagnosed in 2020 [1]. The incidence rate of breast cancer has been increasing over the past few decades, and it is projected to continue rising due to aging populations and changing lifestyles [2]. Despite advances in early detection and treatment, breast cancer remains a complex disease with significant morbidity and mortality.\nOne of the key factors in the development and progression of breast cancer is protein synthesis. Proteins play essential roles in various cellular processes, including growth, differentiation, apoptosis, and metabolism [3]. Dysregulation of protein synthesis can lead to the development of various diseases, including cancer [4]. In breast cancer, abnormalities in protein expression have been implicated in tumorigenesis, progression, and drug resistance [5–7].\nGiven the importance of protein synthesis in breast cancer, it is essential to understand the current state of research in this area. Bibliometric analysis is a powerful tool for evaluating scientific literature and identifying trends and patterns [8]. In this study, we aimed to provide an overview of the current state of research on breast cancer and protein synthesis using bibliometric analysis.\nMethods:\nWe conducted a comprehensive search in Scopus database for articles published between 2015 and 2022 with the keywords “breast cancer,” “protein synthesis,” and related terms, including “translation,” “expression,” “cancer proteins,” and “proteomics.” We excluded reviews, editorials, conference proceedings, and letters to the editor. The search results were analyzed based on various metrics, including publication trends, journal impact factors, most productive countries, authors, and keywords.\nResults:\nA total of 1,538 articles were identified in the search, with an average of 128 articles per year (Figure 1). The number of publications has been increasing steadily over the past few years, indicating a growing interest in this research area.\nThe most productive journals in this field were Journal of Biological Chemistry, Cancer Research, Proceedings of the National Academy of Sciences of the United States of America, and Oncogene (Table 1). These journals have high impact factors and are widely recognized as leaders in the field of cancer research.\nThe most productive countries in terms of research output were the United States, China, and the United Kingdom (Figure 2). The United States accounted for the largest share of publications, followed by China and the United Kingdom.\nThe most productive authors in this field were Xiaohong Wang, Wei Zhang, and Jianhua Wu (Table 2). These authors have published numerous articles on breast cancer and protein synthesis and have made significant contributions to the field.\nKeyword analysis revealed that “breast cancer,” “expression,” “cancer,” “protein,” and “translation” were the most commonly researched topics (Figure 3). These keywords reflect the primary focus of research in this area, which is on understanding the role of protein expression in breast cancer development and progression.\nDiscussion:\nOur bibliometric analysis provides valuable insights into the current state of research on breast cancer and protein synthesis. The increasing number of publications in this field indicates a growing interest in this research area and highlights the importance of continued research to gain a better understanding of the biology of breast cancer and develop new therapies.\nThe most productive journals',
#  'questions_this_excerpt_can_answer': ' 1. Which journals have published the most research articles on breast cancer and protein synthesis between 2015 and 2022, and what are their impact factors?\n
#                                         2. Which countries have contributed the most research articles on breast cancer and protein synthesis during the same time period, and in which order do they rank?\n
#                                         3. Based on the analysis of this bibliometric study, which keywords reflect the primary focus of research in the field of breast cancer and protein synthesis?\n
#                                         4. What is the current state of research on breast cancer and protein synthesis according to the trends and patterns identified through bibliometric analysis?\n
#                                         5. Which authors have made significant contributions to the field of breast cancer and protein synthesis, and how many articles have they published in this area?\n
#                                         6. In what ways can continued research in the field of breast cancer and protein synthesis contribute to a better understanding of the biology of breast cancer and the development of new therapies?',
#  'prev_section_summary': 'This section provides an overview of a bibliometric analysis study on breast cancer and protein synthesis research published between 2015 and 2022. The study identified 1,538 articles with an average of 128 publications per year. The most productive journals were Journal of Biological Chemistry, Cancer Research, Proceedings of the National Academy of Sciences of the United States of America, and Oncogene. Keyword analysis revealed that "breast cancer," "expression," "cancer," "protein," and "translation" were the most commonly researched topics. The study highlights the growing interest in this research area and its importance for understanding breast cancer biology and developing new therapies.\n\nEntities: \n- Breast cancer\n- Protein synthesis\n- Journal of Biological Chemistry\n- Cancer Research\n- Proceedings of the National Academy of Sciences of the United States of America\n- Oncogene\n- Publications\n- Keywords\n- Expression\n- Cancer proteins\n- Proteomics\n- Translation\n\nQuestions: \n1. What was the focus of the bibliometric analysis study on breast cancer and protein synthesis research?\n2. Which journals published the most articles on breast cancer and protein synthesis research between 2015 and 2022?\n3. What were the most commonly researched keywords in breast cancer and protein synthesis literature?\n4. Why is continued research in this area important for understanding breast cancer biology and developing new therapies?',
#  'section_summary': 'This section is an excerpt from a research article that provides an overview of the current state of research on breast cancer and protein synthesis using bibliometric analysis. The authors conducted a comprehensive search in Scopus database for articles published between 2015 and 2022 with the keywords “breast cancer,” “protein synthesis,” and related terms, including “translation,” “expression,” “cancer proteins,” and “proteomics.” They analyzed the results based on various metrics, such as publication trends, journal impact factors, most productive countries, authors, and keywords. The analysis revealed that there has been a significant increase in research publications on breast cancer and protein synthesis over the past few years. The most productive journals were Journal of Biological Chemistry, Cancer Research, Proceedings of the National Academy of Sciences of the United States of America, and Oncogene. Keyword analysis showed that “breast cancer,” “expression,” “cancer,” “protein,” and “translation” were the most commonly researched topics. The authors concluded that continued research in this area is important to gain a better understanding of the biology of breast cancer and develop new therapies.\nKey topics: bibliometric analysis, breast cancer, protein synthesis, publication trends, journal impact factors, most productive countries, authors, keywords.\nEntities: Scopus database, Journal of Biological Chemistry, Cancer Research, Proceedings of the National Academy of Sciences of the United States of America, Oncogene, breast cancer, protein synthesis, expression, cancer, protein, translation.',
#  'excerpt_keywords': 'breast cancer, protein synthesis, bibliometric analysis, publication trends, journal impact factors, most productive countries, authors, keywords, expression, cancer proteins, proteomics, translation.'}



# extract from node 3 originating from page 2
# {'page_label': '2',
#  'file_name': '40001_2023_Article_1364.pdf',
#  'file_path': '/home/dorota/LLM-diploma-project/concept_tests/articles/40001_2023_Article_1364.pdf',
#  'file_type': 'application/pdf',
#  'file_size': 4119241,
#  'creation_date': '2024-05-16',
#  'last_modified_date': '2024-04-02',
#  'document_title': '\nBreast Cancer and Protein Synthesis: Unraveling the Complex Interplay for Effective Therapeutic Interventions through Bibliometric Analysis\n
#                     Abstract: \nBreast cancer is a complex and heterogeneous disease that requires continued research efforts to improve diagnosis, prognosis, and treatment options. Protein synthesis plays a crucial role in breast cancer growth, survival, and metastasis. In this study, we conducted a bibliometric analysis of the literature on protein synthesis and breast cancer to provide an overview of the current state of research in this field. We identified key researchers, institutions, countries, and keywords, as well as trends and patterns in the literature. Our findings suggest that research on protein synthesis and breast cancer is a rapidly growing field, with a focus on hormones, estrogen receptor (ER), progesterone receptor (PR), human epidermal growth factor receptor 2 (HER2), and targeted therapeutic interventions. The most frequent keywords used in the literature include "protein synthesis," "breast cancer," "ribosome," "translation," and "mRNA." The collaboration networks among authors reveal several key collaborations between researchers from different countries, particularly those between the United States and China. Our bibliometric analysis provides valuable insights into the evolution of research on protein synthesis and its relevance to breast cancer, offering a foundation for future research directions in this area.\nKeywords: breast cancer, protein synthesis, hormones, estrogen receptor (ER), progesterone receptor (PR), human epidermal growth factor receptor 2 (HER2), bibliometric analysis.',
#  'questions_this_excerpt_can_answer': '1. Which hormones and proteins have been identified as key players in breast cancer growth, survival, and metastasis through protein synthesis research?\n
#                                           The context discusses the role of hormones such as estrogen receptor (ER) and progesterone receptor (PR), as well as human epidermal growth factor receptor 2 (HER2), in breast cancer. These proteins are crucial for breast tissue development and function, but when overexpressed or abnormally synthesized in breast cancer cells, they can contribute to aggressive and treatment-resistant tumors.\n\n
#                                        2. Which countries have shown significant collaboration in research on protein synthesis and its relevance to breast cancer?\n
#                                           The context mentions several key collaborations between researchers from different countries, particularly those between the United States and China. Bibliometric analysis of the literature reveals these collaborative efforts, offering insights into international research partnerships focused on understanding the complex interplay between protein synthesis and breast cancer.',
#  'entities': ['metastasis'],
#  'prev_section_summary': 'This section is an excerpt from a research article that provides an overview of the current state of research on breast cancer and protein synthesis using bibliometric analysis. The authors conducted a comprehensive search in Scopus database for articles published between 2015 and 2022 with the keywords “breast cancer,” “protein synthesis,” and related terms, including “translation,” “expression,” “cancer proteins,” and “proteomics.” They analyzed the results based on various metrics, such as publication trends, journal impact factors, most productive countries, authors, and keywords. The analysis revealed that there has been a significant increase in research publications on breast cancer and protein synthesis over the past few years. The most productive journals were Journal of Biological Chemistry, Cancer Research, Proceedings of the National Academy of Sciences of the United States of America, and Oncogene. Keyword analysis showed that “breast cancer,” “expression,” “cancer,” “protein,” and “translation” were the most commonly researched topics. The authors concluded that continued research in this area is important to gain a better understanding of the biology of breast cancer and develop new therapies.\nKey topics: bibliometric analysis, breast cancer, protein synthesis, publication trends, journal impact factors, most productive countries, authors, keywords.\nEntities: Scopus database, Journal of Biological Chemistry, Cancer Research, Proceedings of the National Academy of Sciences of the United States of America, Oncogene, breast cancer, protein synthesis, expression, cancer, protein, translation.',
#  'section_summary': 'This excerpt discusses the role of protein synthesis in breast cancer growth, survival, and metastasis, highlighting hormones such as estrogen receptor (ER) and progesterone receptor (PR), as well as human epidermal growth factor receptor 2 (HER2), as key players. The text emphasizes the importance of understanding protein synthesis mechanisms for developing targeted therapeutic interventions, with significant progress made in the past twenty years but much work still to be done. The bibliometric analysis of the literature reveals collaborations between researchers from different countries, particularly the United States and China, in this field. Key entities include breast cancer, hormones (ER, PR), HER2, protein synthesis, ribosome, translation, mRNA, and targeted therapeutic interventions.',
#  'excerpt_keywords': 'breast cancer, hormones, estrogen receptor (ER), progesterone receptor (PR), human epidermal growth factor receptor 2 (HER2), protein synthesis, ribosome, translation, mRNA, targeted therapeutic interventions.'}

In [ ]:
nodes[6].__dict__

In [ ]:
# https://docs.llamaindex.ai/en/stable/module_guides/indexing/metadata_extraction/
# TODO: try custom extractor
# from llama_index.core.extractors import BaseExtractor
# from typing import List, Dict

# class CustomExtractor(BaseExtractor):
#     async def aextract(self, nodes) -> List[Dict]:
#         metadata_list = [
#             {
#                 "custom": node.metadata["document_title"]
#                 + "\n"
#                 + node.metadata["excerpt_keywords"]
#             }
#             for node in nodes
#         ]
#         return metadata_list

## Pydantic Extractor with pipeline and GidancePydanticProgram
https://docs.llamaindex.ai/en/stable/examples/metadata_extraction/PydanticExtractor/

In [ ]:
import nest_asyncio
nest_asyncio.apply()

from pydantic import BaseModel, Field
from typing import List

from llama_index.llms.llama_cpp import LlamaCPP

In [ ]:
class NodeMetadata(BaseModel):
    """Node metadata."""

    entities: List[str] = Field(
        ..., description="Unique entities in this text chunk."
    )
    summary: str = Field(
        ..., description="A concise summary of this text chunk."
    )
    contains_number: bool = Field(
        ...,
        description=(
            "Whether the text chunk contains any numbers (ints, floats, etc.)"
        ),
    )

In [ ]:
llm = LlamaCPP(
    model_path="/home/dorota/models/mistral-7b-instruct-v0.2.Q6_K.gguf",
    context_window=16384, # n_ctx=0
    max_new_tokens=1024,
    model_kwargs={"n_gpu_layers": 33},
    verbose=False
) 

In [ ]:
from llama_index.program.guidance import GuidancePydanticProgram #from llama_index.program.openai import OpenAIPydanticProgram
from llama_index.core.extractors import PydanticProgramExtractor

EXTRACT_TEMPLATE_STR = """\
Here is the content of the section:
----------------
{context_str}
----------------
Given the contextual information, extract out a {class_name} object.\
"""

guidance_program = GuidancePydanticProgram.from_defaults(
    output_cls=NodeMetadata,
    prompt_template_str="{input}",
    # extract_template_str=EXTRACT_TEMPLATE_STR
)

program_extractor = PydanticProgramExtractor(
    program=guidance_program, input_key="input", show_progress=True, extract_template_str = EXTRACT_TEMPLATE_STR
)

In [ ]:
from llama_index.core import SimpleDirectoryReader
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.ingestion import IngestionPipeline

ARTICLE = SimpleDirectoryReader(input_files=["/home/dorota/LLM-diploma-project/00_concept_tests/data/40001_2023_Article_1364.pdf"]).load_data()
ARTICLE = ARTICLE[0:1]

node_parser = SentenceSplitter(chunk_size=1024)
pipeline = IngestionPipeline(transformations=[node_parser, program_extractor])
orig_nodes = pipeline.run(documents=ARTICLE)

# can not run the pipeline with GuidancePydanticProgram; TypeError: GuidancePydanticProgram.program() missing 2 required positional arguments: 'tools_str' and 'query_str'

## Pydantic Extractor with GidancePydanticProgram without pipeline

In [ ]:
from pydantic import BaseModel, Field
from typing import List

In [ ]:
class NodeMetadata(BaseModel):
    """Node metadata."""

    entities: List[str] = Field(
        ..., description="Unique entities in this text chunk."
    )
    summary: str = Field(
        ..., description="A concise summary of this text chunk."
    )
    contains_number: bool = Field(
        ...,
        description=(
            "Whether the text chunk contains any numbers (ints, floats, etc.)"
        ),
    )

In [ ]:
from llama_index.llms.llama_cpp import LlamaCPP
import guidance
llm = guidance.models.LlamaCppChat("/home/dorota/models/mistral-7b-instruct-v0.2.Q6_K.gguf", n_gpu_layers=10,  n_ctx=0)

# llm has to be a guidance model and LlamaCppChat, not just LlamaCPP

In [ ]:
from llama_index.program.guidance import GuidancePydanticProgram
from pypdf import PdfReader 
  
reader = PdfReader('/home/dorota/LLM-diploma-project/00_concept_tests/data/40001_2023_Article_1364.pdf') 
num_pages = len(reader.pages)
TEXT = ""
for page_num in range(1): #change to range(num_pages) for whole document
    page = reader.pages[page_num]  
    TEXT += page.extract_text()


program = GuidancePydanticProgram(
    output_cls=NodeMetadata,
    prompt_template_str=(
        """\
        Here is the content of the section:
        ----------------
        {{query_str}}
        ----------------
        Given the contextual information, extract a pydantic object.\
        """
    ),
    guidance_llm=llm,
    verbose=True,
)

output = program(query_str=TEXT, tools_str='')

# OutputParserException: Failed to parse pydantic object from guidance program. Probably the LLM failed to produce data with right json schema